# Multi-Frame Motion Deblur Recovery
Reconstruction of Resolution Target

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import libwallerlab.projects.motiondeblur as md
from libwallerlab.utilities.io import Dataset

# Debugging
import matplotlib.pyplot as plt
import llops as yp
import ndoperators as ops

# Set backend
ops.setDefaultBackend('arrayfire')

('libforge.1.dylib', '/opt/arrayfire/lib/libforge.1.dylib')


## Load Data

In [2]:
# Define user for path setting
user = 'zack'

if user == 'sarah':
    dataset_path_stem = '/home/sarah/Dropbox/deblurring/motiondeblur_datasets/'
else:
    dataset_path_stem = '/Users/zfphil/Dropbox/Berkeley/Datasets/motiondeblur_datasets/'

# Proces dataet path
dataset_path = 'res_target_color_coded_raster_motiondeblur_2018_05_22_19_17_45'
# dataset_path = 'res_target_color_strobe_raster_motiondeblur_2018_05_22_19_17_18'
# dataset_path = 'res_target_color_stopandstare_raster_multi_contrast_2018_05_22_19_16_26'
filename = dataset_path_stem + dataset_path

# Load dataset
dataset = Dataset(filename, frame_subset=range(19,22))

# Perform motion-deblur related preprocessing
md.preprocess(dataset)

Metadata indicates background has already been subtracted.


## Configure Solver

In [ ]:
dataset.show()

## Create Registration Operator

# Registration

1. Measurement domain

$ f(s) = || A_{x} E \vec{s} - y ||_2^2 $

$\nabla_s f(s) = \nabla E A_{x}^H (A_{x} E\vec{s} - y) $

2. Object domain

$ f(s) = ||x_{k+1} E \vec{s} - x_k ||_2^2 $

$\nabla_s f(s) = \nabla E \times (x_{k+1} E \vec{s} - x_k ) $


## Is phase correlation actually inversion?

$ s^* = \max(|F^H \times (\tilde{y}_1^* \cdot \tilde{y}_2)|)$

$ y_1 = y_2 * F^H \times (\tilde{y}_1^* \cdot \tilde{y}_2) $

$ y_2 = F^H \times |\tilde{y}_1|^2\cdot \tilde{y}_2 $

## Inversion

In [ ]:
recon = md.Reconstruct(dataset)
# recon._get_optimizer()
recon.gen_forward_model() #TODO check the shift (adjust_by=[[0,-313-1024], [0,-313-1024], [0,0]])
# recon.reconstruct(5, objective_type='global') # 20
recon.invert(reg_types={'l2' : 1e-8})
recon.show(figsize=(11,5), clim=False)

## Sequential Testing

In [ ]:
recon = md.Reconstruct(dataset)
recon.gen_forward_model() #TODO check the shift (adjust_by=[[0,-313-1024], [0,-313-1024], [0,0]])
recon._get_optimizer(reg_types={'l2': 1e-6})

In [ ]:
# Get cost function
O = recon.cost_function
y_list = recon.y_list

# TODO: This is a hack
data_term = O.suboperators[0].stack_operators[0]
reg_term = O.suboperators[0].stack_operators[1]

# Strip L2 norm and vector sum
data_term_stripped = data_term._inner_operator._inner_operator

# Compress
data_term_compressed = ops.compressStack(data_term_stripped)

# Get number of measurements in sequence
sequence_count = len(data_term_compressed.stack_operators)

# Form list and build back list of operators
sequential_cost_functions = []
for index, cost_function in enumerate(data_term_compressed.stack_operators):
    # Subtract measurement
    cost_function -= y_list[index]
    
    # Add L2 Norm
    cost_function = ops.L2Norm(cost_function.M, cost_function.dtype, cost_function.backend) * cost_function
    
    # Add regularization (normalize ny number of measurements)
    cost_function += ((1.0 / sequence_count) * reg_term)
    
    # Append to list
    sequential_cost_functions.append(cost_function)


## Global Algorithm

In [ ]:
recon = md.Reconstruct(dataset)
# recon._get_optimizer()
recon.gen_forward_model() #TODO check the shift (adjust_by=[[0,-313-1024], [0,-313-1024], [0,0]])
# recon.reconstruct(5, objective_type='global') # 20
recon._get_optimizer()

In [ ]:
y = recon.cost_function._inner_operator * x

grad = recon.cost_function._inner_operator.H * recon.cost_function._inner_operator * x

In [ ]:
recon.initialization.shape

In [ ]:
recon.cost_function.gradient_check()

In [ ]:
recon.cost_function.suboperators[3].latex()

In [ ]:
Q.N

In [ ]:
Q = recon.cost_function.suboperators[-2]
P = recon.cost_function.suboperators[-1]


x = yp.ones(Q.N)
print(x.shape)
Q._gradient(O=P, x=x)

In [ ]:
recon.cost_function.suboperators[-1]

In [ ]:
Ri = recon.cost_function.suboperators[-1].stack_operators[0]

In [ ]:
Q = recon.cost_function.suboperators[-1]
Q.latex()

# Q.gradient_check()

# print(x.shape)
print(Q.N)
Q._gradient(yp.ones(Q.N))

In [ ]:
Q = recon.cost_function.suboperators[0].stack_operators[0].suboperators[2]
Q.latex()


Q.H * Q * yp.ones(Q.N)

In [ ]:
Q = recon.cost_function.suboperators[0].stack_operators[0].suboperators[2]
Q.latex()

# Try forward operator
Q * yp.ones(Q.N)

# Try adjoint
Q.H * yp.ones(Q.M)

# Try gradient
# Q.gradient_check()




In [ ]:
x = recon.initialization.copy()

step_size = 1e0
n_iterations = 10
for _ in range(n_iterations):
    print(recon.cost_function(x))
    x[:] -= step_size * recon.cost_function.gradient(x)

plt.figure()
plt.imshow(yp.abs(x))

In [ ]:
iterations = [5, 10, 20, 50, 100, 200]
previous = None

for i in range(len(iterations)):
    print('single frame with', iterations[i])
    recon.gen_singleframe_forward_models()
    if previous is not None:
        recon.reconstruct_singeframe(iterations[i]-iterations[i-1], initializations=previous)
    else:
        recon.reconstruct_singeframe(iterations[i])
    previous = recon.objects_recovered_singleframe
    recon.register_singleframe_reconstructions(process=False, debug=True, show_stitched=True)
    # recon.reconstruct_multiframe_from_singleframe(100+200-iterations[i])
    
    recon.reset()

In [ ]:
x = recon.initialization.copy()

step_size = 1e0
n_iterations = 10
for _ in range(n_iterations):
    print(recon.cost_function(x))
    for cost_function in sequential_cost_functions:
        x -= step_size * cost_function.gradient(x)
        
plt.figure()
plt.imshow(yp.abs(x))

In [ ]:
A.operators[0]

In [ ]:
print(A0)
print(_A)
print(_A.operators[0])

In [ ]:
print(A.N)
print(A.operators[0].N)

In [ ]:
A.shape